In [1]:
from fastai.tabular.all import *

In [2]:
train=pd.read_csv("/kaggle/input/playground-series-s4e5/train.csv")
test=pd.read_csv("/kaggle/input/playground-series-s4e5/test.csv")

submission=pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")

original=pd.read_csv("/kaggle/input/flood-prediction-factors/flood.csv")

train.drop(columns=["id"],inplace=True)
test.drop(columns=["id"],inplace=True)

train=pd.concat([train,original],axis='rows')
train.reset_index(inplace=True,drop=True)

target='FloodProbability'

In [3]:
train.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,5,8,5,8,6,4,4,3,3,4,...,5,3,3,5,4,7,5,7,3,0.445
1,6,7,4,4,8,8,3,5,4,6,...,7,2,0,3,5,3,3,4,3,0.450
2,6,5,6,7,3,7,1,5,4,5,...,7,3,7,5,6,8,2,3,3,0.530
3,3,4,6,5,4,8,4,7,6,8,...,2,4,7,4,4,6,5,7,5,0.535
4,5,3,2,6,4,4,3,3,3,3,...,2,2,6,6,4,1,2,3,5,0.415


In [4]:
def meta_features(df, initial_features ):
    
    df[original.drop(columns=[target]).columns]=df[original.drop(columns=[target]).columns].astype(np.uint8)
    df['fsum'] = df[initial_features].sum(axis=1) # for tree models
    print('comnputing f_std')
    df['f_std']  = df[initial_features].std(axis=1)
    print('comnputing f_mean')
    df['f_mean'] = df[initial_features].mean(axis=1)
    print('comnputing f_max')
    df['f_max']  = df[initial_features].max(axis=1)
    print('comnputing f_min')
    df['f_min']  = df[initial_features].min(axis=1)
#     print('comnputing f_mode')
#     df['f_mode'] = df[initial_features].mode(axis=1)[0]
    print('comnputing f_median')
    df['f_median'] = df[initial_features].median(axis=1)
    print('comnputing f_25th')
    df['f_25th'] = df[initial_features].quantile(0.25, axis=1)
    print('comnputing f_75th')
    df['f_75th'] = df[initial_features].quantile(0.75, axis=1)
    print('comnputing f_skew')
    df['f_skew'] = df[initial_features].skew(axis=1)
    print('comnputing f_kurt')
    df['f_kurt'] = df[initial_features].kurt(axis=1)
    df['special1'] = df['fsum'].isin(np.arange(72, 76)) # for linear models
    for i in range(10,100,10):
        print(f'comnputing f_{i}th')
        df[f'f_{i}th'] = df[initial_features].quantile(i/100, axis=1)
    print('comnputing f_harmonic')
    df['f_harmonic'] = len(initial_features) / df[initial_features].apply(lambda x: (1/x).mean(), axis=1)
    print('comnputing f_geometric')
    df['f_geometric'] = df[initial_features].apply(lambda x: x.prod()**(1/len(x)), axis=1)
    print('comnputing f_zscore')
    df['f_zscore'] = df[initial_features].apply(lambda x: (x - x.mean()) / x.std(), axis=1).mean(axis=1)
    print('computing Coefficient of Variation ')
    df['f_cv'] = df[initial_features].std(axis=1) / df[initial_features].mean(axis=1)
    print('computing f_Quantile Coefficients of Skewness_75')
    df['f_Quantile Coefficients of Skewness_75'] = (df[initial_features].quantile(0.75, axis=1) - df[initial_features].mean(axis=1)) / df[initial_features].std(axis=1)
    print('computing f_Quantile Coefficients of Skewness_25')
    df['f_Quantile Coefficients of Skewness_25'] = (df[initial_features].quantile(0.25, axis=1) - df[initial_features].mean(axis=1)) / df[initial_features].std(axis=1)
    print('computing f_2ndMoment')
    df['f_2ndMoment'] = df[initial_features].apply(lambda x: (x**2).mean(), axis=1)
    print('computing f_3rdMoment')
    df['f_3rdMoment'] = df[initial_features].apply(lambda x: (x**3).mean(), axis=1)
    print('computing f_entropy')
    df['f_entropy'] = df[initial_features].apply(lambda x: -1*(x*np.log(x)).sum(), axis=1)
    
    return df

initial_features=test.columns
train=meta_features(train,initial_features )
test =meta_features(test, initial_features)

comnputing f_std
comnputing f_mean
comnputing f_max
comnputing f_min
comnputing f_median
comnputing f_25th
comnputing f_75th
comnputing f_skew
comnputing f_kurt
comnputing f_10th
comnputing f_20th
comnputing f_30th
comnputing f_40th
comnputing f_50th
comnputing f_60th
comnputing f_70th
comnputing f_80th
comnputing f_90th
comnputing f_harmonic
comnputing f_geometric
comnputing f_zscore
computing Coefficient of Variation 
computing f_Quantile Coefficients of Skewness_75
computing f_Quantile Coefficients of Skewness_25
computing f_2ndMoment
computing f_3rdMoment
computing f_entropy


/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


comnputing f_std
comnputing f_mean
comnputing f_max
comnputing f_min
comnputing f_median
comnputing f_25th
comnputing f_75th
comnputing f_skew
comnputing f_kurt
comnputing f_10th
comnputing f_20th
comnputing f_30th
comnputing f_40th
comnputing f_50th
comnputing f_60th
comnputing f_70th
comnputing f_80th
comnputing f_90th
comnputing f_harmonic
comnputing f_geometric
comnputing f_zscore
computing Coefficient of Variation 
computing f_Quantile Coefficients of Skewness_75
computing f_Quantile Coefficients of Skewness_25
computing f_2ndMoment
computing f_3rdMoment
computing f_entropy


/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
def new_features(data):
    data['Land_Use_Pressure'] = data['Urbanization'] + data['Deforestation'] + data['AgriculturalPractices']
    
    data['Environmental_Degradation'] = data['Deforestation'] + data['Siltation'] + data['WetlandLoss'] + data['Landslides']
    
    data['Infrastructure_Vulnerability'] = data['DeterioratingInfrastructure'] + data['InadequatePlanning']
    
    data['Community_Preparedness'] = data['IneffectiveDisasterPreparedness'] + data['PoliticalFactors']
    
    data['Hydrological_Stress'] = data['DrainageSystems'] + data['TopographyDrainage'] + data['Watersheds']
    
    data['Population_Density_Vulnerable_Areas'] = data['PopulationScore'] * data['CoastalVulnerability']
    
    data['Climate_Change_Impact'] = data['ClimateChange'] + data['MonsoonIntensity']
    
    data['River_Health'] = data['RiverManagement'] + data['DamsQuality']
    
    return data

train2=new_features(train)
test2 =new_features(test)

In [6]:
# train.to_csv("/kaggle/working/train_meta_f.csv")
# test.to_csv("/kaggle/working/test_meta_f.csv")

In [7]:
train2.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,f_3rdMoment,f_entropy,Land_Use_Pressure,Environmental_Degradation,Infrastructure_Vulnerability,Community_Preparedness,Hydrological_Stress,Population_Density_Vulnerable_Areas,Climate_Change_Impact,River_Health
0,5,8,5,8,6,4,4,3,3,4,...,70.70,-151.625,17,19,11,5,18,21,9,9
1,6,7,4,4,8,8,3,5,4,6,...,73.60,-158.000,16,12,9,12,17,6,14,7
2,6,5,6,7,3,7,1,5,4,5,...,94.65,-166.500,14,21,9,9,17,24,13,7
3,3,4,6,5,4,8,4,7,6,8,...,91.40,-176.500,15,24,11,10,10,24,11,10
4,5,3,2,6,4,4,3,3,3,3,...,70.50,-98.375,13,17,7,10,11,2,9,5


In [8]:
cont_names,cat_names = cont_cat_split(train2, dep_var='FloodProbability')
splits = RandomSplitter(valid_pct=0.2)(range_of(train2))

to = TabularPandas(train2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='FloodProbability',
                   y_block=RegressionBlock(),
                   splits=splits)

In [9]:
dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test2)

In [10]:
# dls = TabularDataLoaders.from_df(df, y_names="FloodProbability",bs=1024)

In [11]:
# dls.show_batch()

In [12]:
learn = tabular_learner(dls, metrics=[accuracy,R2Score()],lr=0.001)

In [13]:
#  suggest_funcs = (minimum, steep, valley, slide)
#  learn.lr_find(suggest_funcs=suggest_funcs)

In [14]:
learn.fine_tune(15)

epoch,train_loss,valid_loss,accuracy,r2_score,time
0,0.000810,0.000533,0.000000,0.793970,00:17


epoch,train_loss,valid_loss,accuracy,r2_score,time
0,0.000420,0.000417,0.000000,0.839012,00:16
1,0.000425,0.000413,0.000000,0.840362,00:16
2,0.000426,0.000402,0.000000,0.844688,00:16
3,0.000431,0.000401,0.000000,0.845057,00:16
4,0.000423,0.000412,0.000000,0.840745,00:16
5,0.000418,0.000405,0.000000,0.843626,00:16
6,0.000398,0.000406,0.000000,0.843007,00:16
7,0.000393,0.000393,0.000000,0.848204,00:16
8,0.000377,0.000364,0.000000,0.859160,00:16
9,0.000373,0.000370,0.000000,0.856908,00:16


In [15]:
# test_df = pd.read_csv('/kaggle/input/playground-series-s4e5/test.csv',index_col='id')
# dl = learn.dls.test_dl(test_df)

In [16]:
preds = learn.get_preds(dl=test_dl)
preds

(tensor([[0.5766],
         [0.4577],
         [0.4535],
         ...,
         [0.6193],
         [0.5531],
         [0.5280]]),
 None)

In [17]:
preds1 = preds[0].cpu().numpy()

In [18]:
to = TabularPandas(train2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='FloodProbability',
                   y_block=RegressionBlock(),
                   splits=splits)

In [19]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [20]:
from sklearn.ensemble import RandomForestRegressor
def rf(xs, y, n_estimators=40, max_samples=200000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
        max_samples=max_samples, max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [21]:
from sklearn.metrics import r2_score

def cal_score(m,X,y): 
    preds = m.predict(X)
    return r2_score(y,preds)

In [22]:
m = rf(X_train, y_train,n_estimators=80,max_samples=200000,
       max_features=0.4, min_samples_leaf=4);
score = cal_score(m,X_test,y_test)
score

0.8691400811017018

In [23]:
preds2 = m.predict(test_dl.xs)
preds2

array([0.57887496, 0.45434502, 0.44339551, ..., 0.61959177, 0.54792682,
       0.52577487])

In [24]:
from xgboost import XGBRegressor

xgb = XGBRegressor(
    max_depth=13,
    learning_rate=0.22472048294428892,
    n_estimators=767,
    min_child_weight=10,
    gamma=8.58031910185836e-07,
    subsample=0.8936985751131682,
    colsample_bytree=0.04050040446078567,
    reg_alpha=0.16025160863896398,
    reg_lambda=6.875124639971543e-08,
)

xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.04050040446078567, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=8.58031910185836e-07,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.22472048294428892,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=13, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=767, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [25]:
score = cal_score(xgb,X_test,y_test)
score

0.8670780782119639

In [26]:

from catboost import CatBoostRegressor

cat = CatBoostRegressor(
    iterations=950,
    learning_rate=0.07024611544300416,
    colsample_bylevel=0.32374524982071545,
    min_data_in_leaf=69,
    depth=10,
    l2_leaf_reg=5.938240087759686,
)

cat.fit(X_train, y_train)

0:	learn: 0.0480079	total: 322ms	remaining: 5m 5s
1:	learn: 0.0453262	total: 577ms	remaining: 4m 33s
2:	learn: 0.0427938	total: 838ms	remaining: 4m 24s
3:	learn: 0.0404461	total: 1.13s	remaining: 4m 26s
4:	learn: 0.0383024	total: 1.42s	remaining: 4m 28s
5:	learn: 0.0363735	total: 1.67s	remaining: 4m 23s
6:	learn: 0.0345709	total: 1.92s	remaining: 4m 18s
7:	learn: 0.0329386	total: 2.16s	remaining: 4m 14s
8:	learn: 0.0314475	total: 2.42s	remaining: 4m 13s
9:	learn: 0.0300936	total: 2.68s	remaining: 4m 11s
10:	learn: 0.0288581	total: 2.94s	remaining: 4m 11s
11:	learn: 0.0277505	total: 3.21s	remaining: 4m 10s
12:	learn: 0.0267646	total: 3.46s	remaining: 4m 9s
13:	learn: 0.0258635	total: 3.72s	remaining: 4m 8s
14:	learn: 0.0250348	total: 3.98s	remaining: 4m 8s
15:	learn: 0.0243061	total: 4.23s	remaining: 4m 6s
16:	learn: 0.0236602	total: 4.46s	remaining: 4m 4s
17:	learn: 0.0230651	total: 4.71s	remaining: 4m 3s
18:	learn: 0.0225434	total: 4.96s	remaining: 4m 2s
19:	learn: 0.0220846	total: 5.

In [27]:
score = cal_score(xgb,X_test,y_test)
score

0.8670780782119639

In [28]:
preds3 = xgb.predict(test_dl.xs)
preds3

array([0.579288  , 0.45659697, 0.44622296, ..., 0.6241868 , 0.544679  ,
       0.53110665], dtype=float32)

In [29]:
preds4 = cat.predict(test_dl.xs)
preds4

array([0.57799884, 0.45795943, 0.44692161, ..., 0.62169026, 0.54820514,
       0.52856406])

In [30]:
preds1 = preds1.squeeze()

In [31]:

# preds_en = np.stack((preds1,preds2,preds3,preds4))
# preds_en = np.mean(preds_en,axis=0)
# preds_en

In [32]:
sub = pd.read_csv("/kaggle/input/playground-series-s4e5/sample_submission.csv")
sub['FloodProbability'] = (preds1 + preds2 + preds3 + preds4) /4
sub.head()

,id,FloodProbability
0,1117957,0.578181
1,1117958,0.456657
2,1117959,0.447504
3,1117960,0.464943
4,1117961,0.466294


In [33]:
sub.to_csv("/kaggle/working/submission_en.csv",index=False)

In [34]:
# def rf_feat_importance(m, df):
#     return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
#                        ).sort_values('imp', ascending=False)

In [35]:
# fi = rf_feat_importance(m, X_test)
# fe=fi[:15].cols.to_list()
# fe

In [36]:
# def plot_fi(fi):
#     return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

# plot_fi(fi[:15]);